In [1]:
import torch
import numpy as np
from utils.dataset import SimpleIterDataset
import os
import onnxruntime as ort

In [2]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [3]:
def create_ort_inputs(inputs, input_names):
    return {name: to_numpy(inputs[index]) for (index, name) in enumerate(input_names)}

In [4]:
def run_ort_inference(onnx_path, inputs, input_names):
    ort_session = ort.InferenceSession(onnx_path)
    runSessionParams = ort.RunOptions()
    runSessionParams.log_severity_level = 2
    ort_out = ort_session.run(None, create_ort_inputs(inputs, input_names), run_options = runSessionParams)
    return ort_out

In [5]:
def run_pt_inference(model, inputs):
    return model(*inputs)

In [6]:
def export_onnx(model, inputs, dynamic_axes, input_names, output_names, onnx_path, opset_version = 15):
    torch.onnx.export(model, inputs, onnx_path, input_names=input_names, output_names=output_names, dynamic_axes=dynamic_axes, opset_version=opset_version)
    model_output = run_pt_inference(model, inputs)
    ort_output = run_ort_inference(onnx_path, inputs, input_names)
    print ("Model output - \n{}".format(model_output))
    print ("ORT output - \n{}".format(ort_output))

In [7]:
def create_model_inputs(model_type, num_jets = 4, num_particles_per_jet = 1, device = 'cpu', expectedIntDtype = 'int32'):
    def get_feature_dimension(feature_name, model_types_dims_mapper, default_dims_mapper, model_type):
        dimensions = None
        if feature_name in default_dims_mapper.keys():
            dimensions = default_dims_mapper[feature_name]
        else:
            dimensions = model_types_dims_mapper[model_type][feature_name]
        return dimensions
    
    model_type_dims_mapper = {'ak8_points_pf_sv_full': {'pf_features': 25, 'sv_features': 11}, 'ak8_points_pf_sv_mass_decorr': {'pf_features': 20, 'sv_features': 11}, 'ak4_points_pf_sv_CHS_eta4p7': {'pf_features': 20, 'sv_features': 11}, 'ak8_points_pf_sv_mass_regression': {'pf_features': 25, 'sv_features': 11}}
    default_dims_mapper = {'pf_points': 2, 'pf_mask': 1, 'sv_points': 2, 'sv_mask': 1}
    input_names = ['pf_points', 'pf_features', 'pf_mask', 'sv_points', 'sv_features', 'sv_mask']
    input_names += ['batch_shapes_{}'.format(item) for item in input_names]
    
    inputs = list()
    for input_name in input_names:
        if 'batch_shapes_' not in input_name:
            dimensions = get_feature_dimension(input_name, model_type_dims_mapper, default_dims_mapper, model_type)    
            inputs.append(torch.rand((num_jets * num_particles_per_jet * dimensions, ), dtype = torch.float).to(device))
        else:
            dimensions = get_feature_dimension(input_name.replace('batch_shapes_', ''), model_type_dims_mapper, default_dims_mapper, model_type)    
            datatype = torch.int32 if expectedIntDtype == 'int32' else torch.int64
            inputs.append(torch.tensor([[dimensions, num_particles_per_jet] for _ in range(num_jets)], dtype = datatype).to(device))
    return tuple(inputs), input_names

In [8]:
def get_model(model_type, device = 'cpu'):
    data_config_name = 'data/AK4/{}.yaml'.format(model_type)
    if model_type != 'ak4_points_pf_sv_CHS_eta4p7':
        data_config_name = 'data/{}.yaml'.format(model_type)
    import networks.particle_net_ak4_pf_sv as network_module1
    import networks.particle_net_pf_sv as network_module2
    import networks.particle_net_pf_sv_mass_regression as network_module3
    
    model_state_dict = data_config_name.replace('.yaml','.pt')
    jit_model_save = data_config_name.replace('.yaml','_ragged_gpu_jit.pt')
    onnx_model = data_config_name.replace('.yaml','.onnx')
    
    data_config = SimpleIterDataset([], data_config_name, for_training=False).config
    model, model_info = None, None
    
    network_module = None
    if model_type == 'ak8_points_pf_sv_full':
        network_module = network_module2
    elif model_type == 'ak8_points_pf_sv_mass_decorr':
        network_module = network_module2
    elif model_type == 'ak4_points_pf_sv_CHS_eta4p7':
        network_module = network_module1
    else:
        network_module = network_module3
    
    model, model_info = network_module.get_model(data_config, for_inference=True)
    model = torch.jit.script(model)
    model.load_state_dict(torch.load(model_state_dict, map_location=torch.device(device)))
    model.eval()
    
    return model, model_info

In [9]:
def export_multiple_models(model_types):
    models = list() 
    for model_type in model_types:
        print ("Export model type - {}".format(model_type)) 
        model, model_info = get_model(model_type)
        model_inputs, input_names = create_model_inputs(model_type, num_jets = 9, num_particles_per_jet = 3)
        onnx_path = "exported_models/{}.onnx".format(model_type)
        export_onnx(model, model_inputs, model_info['dynamic_axes'], input_names, model_info['output_names'], onnx_path)
        models.append(model)
    return models

In [10]:
def test_exported_models(models, model_types, configs = [{'num_jets': 1, 'num_particles_per_jet': 13}, {'num_jets': 1, 'num_particles_per_jet': 1}, {'num_jets': 17, 'num_particles_per_jet': 3}, {'num_jets': 16, 'num_particles_per_jet': 20}]):
    for index, model_type in enumerate(model_types):
        model = models[index]
        onnx_path = "exported_models/{}.onnx".format(model_type)
        print ("Model Type - {}\n".format(model_type))
        print ("------------")
        for config in configs:
            print ("Config - Num Jets = {} | Num particles per jet = {}".format(config['num_jets'], config['num_particles_per_jet']))
            test_model_inputs, test_model_input_names = create_model_inputs(model_type, **config)
            print ("PT inference - \n{}\n".format(run_pt_inference(model, test_model_inputs)))
            print ("ORT inference - \n{}\n".format(run_ort_inference(onnx_path, test_model_inputs, test_model_input_names)))

In [11]:
#Test PT model
model_type = 'ak8_points_pf_sv_full'
model_inputs, names = create_model_inputs(model_type)
run_pt_inference(get_model(model_type)[0], model_inputs)

tensor([[9.9208e-01, 3.5801e-04, 3.2226e-05, 3.0899e-08, 6.3638e-09, 5.5457e-08,
         1.8243e-03, 2.8271e-04, 8.1947e-08, 8.5341e-04, 2.2260e-05, 1.2879e-07,
         1.9996e-03, 4.3198e-05, 3.3491e-04, 2.1666e-03, 2.6601e-06, 5.5280e-07,
         9.8616e-11, 1.6854e-16],
        [9.3783e-01, 1.1451e-05, 7.2551e-03, 3.9257e-07, 1.9478e-09, 3.3066e-05,
         3.4399e-03, 1.1821e-02, 5.8585e-08, 3.0253e-04, 6.0421e-07, 2.2767e-10,
         5.8198e-03, 3.2639e-04, 1.9109e-02, 1.4045e-02, 1.6632e-06, 1.3285e-06,
         1.4630e-11, 9.1555e-20],
        [8.2083e-02, 6.2414e-02, 2.5899e-04, 4.4329e-05, 1.5275e-06, 6.1414e-11,
         9.9265e-02, 8.7922e-09, 1.3300e-10, 1.7062e-04, 7.7889e-06, 2.1681e-10,
         6.7002e-01, 2.4259e-03, 7.6922e-02, 6.2129e-03, 1.1962e-05, 1.6366e-04,
         3.8962e-11, 3.1072e-21],
        [1.0079e-03, 7.3534e-08, 8.7776e-10, 8.6493e-13, 4.6508e-23, 2.2074e-14,
         1.4188e-05, 3.2068e-07, 1.3446e-10, 1.0645e-05, 4.1524e-08, 6.4263e-10,
       

In [12]:
model_types = ['ak8_points_pf_sv_full', 'ak4_points_pf_sv_CHS_eta4p7', 'ak8_points_pf_sv_mass_decorr', 'ak8_points_pf_sv_mass_regression']
models = export_multiple_models(model_types)

Export model type - ak8_points_pf_sv_full


/Users/nirmalthomas/Library/Python/3.8/lib/python/site-packages/torch/onnx/symbolic_helper.py:719: UserWarning: allowzero=0 by default. In order to honor zero value in shape use allowzero=1
  warnings.warn("allowzero=0 by default. In order to honor zero value in shape use allowzero=1")
2022-08-02 20:45:47.110028 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:45:47.126260 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:47.126274 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:47.126281 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not u

Model output - 
tensor([[8.5396e-01, 4.4762e-04, 8.4531e-06, 1.0558e-08, 2.5307e-07, 1.2812e-05,
         3.7738e-02, 3.8432e-05, 3.4099e-10, 2.4257e-02, 3.9522e-05, 1.0037e-08,
         4.5330e-02, 3.0871e-05, 1.6279e-02, 2.1849e-02, 3.9206e-06, 4.4708e-06,
         1.1786e-12, 2.3452e-24],
        [4.4913e-01, 6.6978e-03, 1.9181e-03, 1.8567e-04, 3.5931e-06, 3.6394e-02,
         1.7986e-01, 1.9027e-02, 1.9602e-05, 1.5800e-02, 3.2526e-03, 1.7824e-05,
         3.2105e-02, 2.8845e-03, 2.4913e-01, 3.5242e-03, 1.9961e-05, 2.1311e-05,
         5.3273e-09, 1.0053e-13],
        [2.6818e-01, 4.3709e-02, 2.2713e-05, 2.1409e-06, 2.2403e-08, 5.3475e-05,
         9.7054e-02, 1.3653e-09, 2.5183e-10, 1.4863e-03, 4.6540e-10, 7.2098e-10,
         3.7138e-01, 3.1737e-08, 1.8377e-01, 3.4347e-02, 4.7519e-11, 1.0308e-06,
         5.2877e-19, 9.9712e-26],
        [6.1704e-02, 2.3860e-02, 1.6936e-03, 6.4505e-04, 1.7379e-03, 3.6792e-02,
         8.0800e-01, 2.2512e-06, 4.8250e-07, 6.6018e-03, 1.6681e-06, 2.6

2022-08-02 20:45:49.794140 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:45:49.810991 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:49.811007 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:49.811013 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:49.811019 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:49.811024 [W:onnxruntime:, gr

Model output - 
tensor([[5.4242e-02, 2.6094e-01, 2.9431e-02, 4.4001e-02, 5.4686e-03, 2.0992e-01,
         3.9599e-01, 1.3027e-07],
        [6.9841e-03, 9.9152e-01, 9.7432e-05, 4.5907e-04, 2.7181e-07, 5.5412e-05,
         8.8537e-04, 1.7270e-24],
        [2.4976e-01, 2.0081e-01, 8.0664e-02, 4.0738e-02, 1.0853e-02, 1.8866e-01,
         2.2752e-01, 9.9937e-04],
        [1.3596e-01, 2.6010e-01, 4.9985e-02, 3.4294e-02, 2.9337e-03, 1.4221e-01,
         3.7452e-01, 3.5834e-09],
        [1.4825e-01, 1.0894e-01, 6.8103e-02, 3.5396e-02, 2.1058e-02, 3.0745e-01,
         3.1080e-01, 1.0780e-06],
        [1.5781e-02, 1.3842e-02, 1.3438e-03, 3.9645e-04, 2.0902e-04, 4.5438e-03,
         9.4266e-03, 9.5446e-01],
        [1.4623e-01, 1.3251e-01, 7.6268e-02, 4.2561e-02, 1.2807e-02, 3.1218e-01,
         2.7514e-01, 2.2928e-03],
        [1.6859e-02, 9.8043e-01, 1.7296e-04, 8.8402e-04, 4.6677e-07, 1.8399e-04,
         1.4732e-03, 4.4199e-16],
        [2.2279e-02, 8.3414e-01, 4.4059e-03, 1.2906e-02, 3.8915e

2022-08-02 20:45:52.916652 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:45:52.933079 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:52.933095 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:52.933102 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:52.933107 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:52.933113 [W:onnxruntime:, gr

Model output - 
tensor([[5.6486e-11, 9.3758e-11, 6.4973e-23, 1.0000e+00, 5.2284e-09, 2.5922e-11,
         9.1718e-15, 7.7543e-23],
        [3.8448e-13, 7.6233e-13, 8.8697e-31, 1.0000e+00, 9.2576e-17, 1.1266e-15,
         1.0470e-26, 9.5221e-33],
        [2.3470e-18, 1.4142e-13, 9.6375e-29, 1.0000e+00, 2.5258e-11, 3.1396e-13,
         1.8645e-22, 4.8984e-33],
        [3.1916e-11, 1.6189e-12, 7.6795e-24, 1.0000e+00, 4.3841e-08, 2.1851e-07,
         2.6073e-13, 1.6856e-21],
        [1.0719e-09, 1.8077e-08, 7.0191e-18, 1.0000e+00, 2.8324e-10, 3.2965e-09,
         6.9987e-17, 5.3844e-20],
        [1.9157e-07, 8.0636e-10, 3.5047e-28, 1.0000e+00, 8.5574e-14, 2.0138e-11,
         6.3960e-24, 6.7067e-34],
        [8.3911e-18, 1.6846e-15, 1.3890e-30, 1.0000e+00, 4.3371e-10, 1.3231e-10,
         3.7104e-17, 1.8748e-33],
        [1.1365e-15, 3.2694e-15, 4.0158e-33, 1.0000e+00, 1.0929e-22, 1.7499e-12,
         1.0446e-35, 0.0000e+00],
        [3.6350e-14, 4.8367e-09, 1.3602e-16, 1.0000e+00, 5.7239e

2022-08-02 20:45:56.015402 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:45:56.032490 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:56.032508 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:56.032514 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:56.032520 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:56.032526 [W:onnxruntime:, gr

In [13]:
test_exported_models(models, model_types)

Model Type - ak8_points_pf_sv_full

------------
Config - Num Jets = 1 | Num particles per jet = 13
PT inference - 
tensor([[6.2762e-01, 2.4294e-01, 1.1028e-03, 1.6587e-03, 4.7235e-05, 9.3204e-03,
         3.8988e-02, 7.7117e-03, 6.3684e-04, 6.0335e-04, 3.6322e-04, 9.9051e-05,
         5.5330e-03, 2.5246e-03, 5.5951e-02, 4.4491e-03, 4.2780e-04, 2.1616e-05,
         1.2838e-06, 1.5825e-06]], grad_fn=<SoftmaxBackward0>)



2022-08-02 20:45:57.121262 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:45:57.138301 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:57.138318 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:57.138325 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:57.138330 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:57.138336 [W:onnxruntime:, gr

ORT inference - 
[array([[6.2761647e-01, 2.4294288e-01, 1.1028134e-03, 1.6586527e-03,
        4.7234760e-05, 9.3203709e-03, 3.8987659e-02, 7.7116643e-03,
        6.3683791e-04, 6.0335035e-04, 3.6321278e-04, 9.9050092e-05,
        5.5329991e-03, 2.5246202e-03, 5.5950753e-02, 4.4491175e-03,
        4.2779706e-04, 2.1615504e-05, 1.2838378e-06, 1.5825246e-06]],
      dtype=float32)]

Config - Num Jets = 1 | Num particles per jet = 1
PT inference - 
tensor([[1.3277e-01, 1.0517e-03, 3.0120e-01, 4.9446e-01, 7.5892e-04, 6.1658e-02,
         6.5323e-03, 1.3072e-03, 5.3140e-05, 9.3049e-06, 9.7914e-06, 1.0962e-04,
         7.9100e-05, 2.6963e-06, 1.7280e-06, 7.5194e-07, 1.8486e-10, 7.0569e-07,
         1.1496e-09, 5.1858e-10]], grad_fn=<SoftmaxBackward0>)



2022-08-02 20:45:58.160480 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:45:58.177057 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:58.177072 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:58.177079 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:58.177084 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-08-02 20:45:58.177090 [W:onnxruntime:, gr

ORT inference - 
[array([[1.3277404e-01, 1.0516784e-03, 3.0119511e-01, 4.9445587e-01,
        7.5891789e-04, 6.1658066e-02, 6.5323035e-03, 1.3072097e-03,
        5.3139822e-05, 9.3048784e-06, 9.7914253e-06, 1.0962389e-04,
        7.9099460e-05, 2.6962805e-06, 1.7280139e-06, 7.5193657e-07,
        1.8485159e-10, 7.0568279e-07, 1.1496062e-09, 5.1857746e-10]],
      dtype=float32)]

Config - Num Jets = 17 | Num particles per jet = 3
PT inference - 
tensor([[4.3435e-02, 5.3486e-06, 7.9319e-11, 8.7408e-15, 3.1958e-24, 7.1643e-16,
         1.9543e-04, 5.0750e-09, 6.2599e-13, 1.1873e-04, 1.4750e-09, 3.3469e-11,
         1.1898e-01, 1.2434e-06, 8.3693e-01, 3.3111e-04, 1.4773e-12, 3.5463e-11,
         5.9173e-24, 6.0376e-39],
        [2.0903e-01, 2.3691e-03, 6.3707e-04, 2.2469e-05, 5.3293e-06, 3.9132e-06,
         6.6491e-03, 7.2745e-02, 3.1417e-05, 2.9404e-01, 2.6805e-02, 2.8694e-04,
         1.8908e-01, 9.8338e-03, 1.7593e-01, 1.2318e-02, 5.1630e-05, 1.6202e-04,
         2.4158e-08, 3.1505e-2

...

2022-08-02 20:46:14.081918 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:46:14.099238 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:14.099255 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:14.099262 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:14.099267 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:14.099273 [W:onnxruntime:, gr

ORT inference - 
[array([[1.]], dtype=float32)]

Config - Num Jets = 17 | Num particles per jet = 3
PT inference - 
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SoftmaxBackward0>)



2022-08-02 20:46:15.345655 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:46:15.361736 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:15.361753 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:15.361760 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:15.361765 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:15.361771 [W:onnxruntime:, gr

ORT inference - 
[array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)]

Config - Num Jets = 16 | Num particles per jet = 20
PT inference - 
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SoftmaxBackward0>)

ORT inference - 
[array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)]



2022-08-02 20:46:17.470309 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_447'
2022-08-02 20:46:17.486889 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '170'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:17.486905 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '169'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:17.486911 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '173'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:17.486917 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-08-02 20:46:17.486922 [W:onnxruntime:, gr